In [ ]:
# Задание вытащить все вакансии СБЕРа с ХХ
# Вытащите все описания этих вакансий
# Создайте DataFrame только добавьте поле skills
# Переведите даты публикаций в datetime
# Постройте график опубликованных вакансий по датам
# Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
# Найдите вакансии с использованием python
# Определите по полю skills какие навыки больше всего востребованы для этих вакансий, и
# Постройте график наиболее востребованных навыков

In [17]:
import requests
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd

In [18]:
sber = '3529'
page = 1
num_per_page = 100
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'
res = requests.get(url)
vacancies = res.json()
num_pages = vacancies.get('pages')
num_pages
vacancies = res.json()
num_pages = vacancies.get('pages')
vacancy_ids = [el.get('id') for el in vacancies.get('items')]
all_vacancy_ids= []
for i in tqdm(range(num_pages)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

In [26]:
len(all_vacancy_ids)

1141

In [15]:
vac_id = 43326020
url = f'https://api.hh.ru/vacancies/{vac_id}'
res = requests.get(url)
vacancy = res.json()


In [28]:
tabdict = defaultdict(list)
for vac_id in tqdm(all_vacancy_ids):
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url)
    vacancy = res.json()
    tabdict['id'].append(vacancy.get('id'))
    tabdict['name'].append(vacancy.get('name'))
    tabdict['description'].append(vacancy.get('description'))
    tabdict['skills'].append(vacancy.get('key_skills'))
    tabdict['published_at'].append(vacancy.get('published_at'))


  0%|          | 0/1141 [00:00<?, ?it/s]

In [ ]:
sk = tabdict.get('skills')
tabdict['skills'] = [','.join([y.get('name') for y in x]) for x in sk]

In [32]:
df = pd.DataFrame(tabdict)

In [33]:
df.head()

,id,name,description,skills,published_at
0,55415805,Менеджер по работе с клиентами,<p>Сбер — лучший работодатель России. У нас бо...,[],2022-09-07T13:55:11+0300
1,69355530,Финансовый консультант,<p>Сбер — лучший работодатель России. У нас бо...,[],2022-09-05T11:25:49+0300
2,45381514,Менеджер по привлечению корпоративных клиентов,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Холодные продажи'}, {'name': 'Поиск...",2022-09-01T14:20:50+0300
3,45790891,Менеджер по работе с ключевыми клиентами малог...,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Прямые продажи'}, {'name': 'Финансо...",2022-09-13T09:25:39+0300
4,69957849,Стажёр в отдел документооборота SberDevices,<p>Цифровые поверхности Салют (SberDevices) — ...,[{'name': 'Работа с большим объемом информации...,2022-09-19T11:20:47+0300


In [37]:
class VacancyLoader:
    def __init__(self):
        self.vacancy_df = None
        self.vacancies_ids = []
        self.sber = '3529'
        self.page = 1
        self.num_per_page = 100
        self.moscow = 1
        self.tabdict = defaultdict(list)

    def get_ids(self):
        url = f'https://api.hh.ru/vacancies?employer_id={self.sber}&page={self.page}&per_page={self.num_per_page}&area={self.moscow}'
        res = requests.get(url)
        vacancies = res.json()
        num_pages = vacancies.get('pages')
        for i in tqdm(range(num_pages)):
            url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
            res = requests.get(url)
            vacancies = res.json()
            vacancy_ids = [el.get('id') for el in vacancies.get('items')]
            self.vacancies_ids.extend(vacancy_ids)
        print("ID вакансий собраны!")

    def get_vacancies(self):

        for vac_id in tqdm(self.vacancies_ids):
            url = f'https://api.hh.ru/vacancies/{vac_id}'
            res = requests.get(url)
            vacancy = res.json()
            self.tabdict['id'].append(vacancy.get('id'))
            self.tabdict['name'].append(vacancy.get('name'))
            self.tabdict['description'].append(vacancy.get('description'))
            self.tabdict['skills'].append(vacancy.get('key_skills'))
            self.tabdict['published_at'].append(vacancy.get('published_at'))
        print("Вакансии собраны в словарь")

    def make_vacancy_dataframe(self):
        print("Создан DataFrame")
        return pd.DataFrame(self.tabdict)

    def get_df_with_vacancies(self):
        self.get_ids()
        self.get_vacancies()
        self.vacancy_df = self.make_vacancy_dataframe()
        return self.vacancy_df

In [39]:
vac_loader = VacancyLoader()
vdf = vac_loader.get_df_with_vacancies()

  0%|          | 0/12 [00:00<?, ?it/s]

ID вакансий собраны!


  0%|          | 0/1141 [00:00<?, ?it/s]

Вакансии собраны в словарь
Создан DataFrame


In [40]:
vdf

,id,name,description,skills,published_at
0,55415805,Менеджер по работе с клиентами,<p>Сбер — лучший работодатель России. У нас бо...,[],2022-09-07T13:55:11+0300
1,69355530,Финансовый консультант,<p>Сбер — лучший работодатель России. У нас бо...,[],2022-09-05T11:25:49+0300
2,45381514,Менеджер по привлечению корпоративных клиентов,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Холодные продажи'}, {'name': 'Поиск...",2022-09-01T14:20:50+0300
3,45790891,Менеджер по работе с ключевыми клиентами малог...,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Прямые продажи'}, {'name': 'Финансо...",2022-09-13T09:25:39+0300
4,69957849,Стажёр в отдел документооборота SberDevices,<p>Цифровые поверхности Салют (SberDevices) — ...,[{'name': 'Работа с большим объемом информации...,2022-09-19T11:20:47+0300
...,...,...,...,...,...
1136,68787347,Старший клиентский менеджер,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Деловая переписка'}, {'name': 'Дело...",2022-09-19T09:10:17+0300
1137,67799537,Клиентский менеджер прямых продаж,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Проведение презентаций'}, {'name': ...",2022-09-19T09:11:17+0300
1138,67799534,Клиентский менеджер прямых продаж,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Проведение презентаций'}, {'name': ...",2022-09-19T09:11:16+0300
1139,69364984,Клиентский менеджер прямых продаж,<p>Сбер — лучший работодатель России. У нас бо...,"[{'name': 'Проведение презентаций'}, {'name': ...",2022-09-19T09:09:11+0300


In [49]:
class DataFrameBeautifier:
    def __init__(self, df):
        self.df = df

    def parse_skills(self):
        pass

    def check_datetime_published_at(self):
        pass

In [ ]:
class SkillsVisualizer:
    pass